# CNN basique pour reconnaître les champions de League of Legends

In [7]:
import os
import cv2 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [3]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop, Adam, Adadelta
from keras.utils.np_utils import to_categorical

2023-02-07 19:47:23.068720: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


> Charger les images et assigner les labels

In [8]:
from sklearn import preprocessing

In [9]:
nom_champions_gen = os.listdir('./images_draft')
noms_champions_gen_modif = [nom_champions_gen[x][:-6] for x in range(len(nom_champions_gen))]
# Enlever .DS
#noms_champions_gen_modif = [x for x in noms_champions_gen_modif if x != '.DS']
#print(noms_champions_gen_modif)
#for champion in noms_champions_gen_modif:
#    print(champion)
   # Create a folder for each champion and move the unique image of the champion in it
#    os.mkdir('./images_draft/'+champion)

       


In [55]:
# Move in each folder the file 'the picture
print(noms_champions_gen_modif)
for champion in noms_champions_gen_modif:
    print(champion)
    os.rename('./images_draft/'+champion+'_0.jpg', './images_draft/'+champion+'/'+champion+'_0.jpg')

['Urgot', 'Jinx', 'Galio', 'Corki', 'Rengar', 'Shen', 'Braum', 'Rammus', 'KSante', 'Sylas', 'FiddleSticks', 'Nilah', 'Renekton', 'Twitch', 'Yasuo', 'Vex', 'Camille', 'Garen', 'Thresh', 'Anivia', 'Caitlyn', 'Kayn', 'Viego', 'Kassadin', 'Talon', 'Sejuani', 'Tristana', 'Gangplank', 'Singed', 'LeeSin', 'TwistedFate', 'Khazix', 'Lillia', 'Akali', 'Rell', 'Samira', 'Brand', 'Warwick', 'Alistar', 'Velkoz', 'Syndra', 'Riven', 'Neeko', 'Evelynn', 'Nidalee', 'JarvanIV', 'Orianna', 'Kled', 'MonkeyKing', 'Annie', 'Leblanc', 'Janna', 'Heimerdinger', 'Swain', 'Jhin', 'Jax', 'Soraka', 'Mordekaiser', 'Fizz', 'Morgana', 'Hecarim', 'Nunu', 'Poppy', 'Sion', 'Ziggs', 'Malphite', 'Lucian', 'Xayah', 'Shaco', 'Shyvana', 'Gwen', 'Sett', 'Sivir', 'Graves', 'Irelia', 'Kaisa', 'Ezreal', 'Kayle', 'Kennen', 'Zyra', 'Nami', 'Akshan', 'Zed', 'KogMaw', 'Kalista', 'Cassiopeia', 'Nocturne', 'Quinn', 'Viktor', 'Ryze', 'Zoe', 'Teemo', 'Vi', 'Xerath', 'Amumu', 'Aphelios', 'Jayce', 'Bard', 'Diana', 'Gnar', 'Volibear', 'Udy

In [74]:
# For each folder in the folder images_draft, create duplicates in order to have 5 pictures per champion
for champion in noms_champions_gen_modif:
    for i in range(6, 11):
        # We don't use rename because we want to keep the original picture
        os.system('cp ./images_draft/'+champion+'/'+champion+'_0.jpg ./images_draft/'+champion+'/'+champion+'_'+str(i)+'.jpg')
            


In [10]:
data_path = "./images_draft"
data_dir_list = os.listdir(data_path)
for x in data_dir_list:
    if x == '.DS_Store':
        data_dir_list.remove(x)
print(data_dir_list)
num_classes = len(data_dir_list) 
print(num_classes)

['LeeSin', 'Rakan', 'Diana', 'Vayne', 'MasterYi', 'Nilah', 'Sion', 'Zed', 'Talon', 'Akshan', 'Zoe', 'Akali', 'Bard', 'Riven', 'Vi', 'Ashe', 'Syndra', 'Rell', 'Gragas', 'Darius', 'Malzahar', 'Soraka', 'Gwen', 'Graves', 'Qiyana', 'Azir', 'Viego', 'Ahri', 'Kayle', 'Veigar', 'Rammus', 'FiddleSticks', 'Morgana', 'TahmKench', 'Lillia', 'Jinx', 'Thresh', 'Warwick', 'Nasus', 'Fiora', 'Ziggs', 'Nautilus', 'Yasuo', 'Khazix', 'Sylas', 'Ivern', 'Kaisa', 'Shyvana', 'Nami', 'Orianna', 'Ezreal', 'Malphite', 'Mordekaiser', 'Brand', 'Leona', 'Shaco', 'Ryze', 'Nocturne', 'Rengar', 'Kled', 'Senna', 'Varus', 'Jhin', 'Singed', 'MonkeyKing', 'Aatrox', 'Vladimir', 'Belveth', 'Heimerdinger', 'Zilean', 'Skarner', 'Yuumi', 'Samira', 'Lissandra', 'Pantheon', 'Caitlyn', 'Braum', 'Kennen', 'Zac', 'Illaoi', 'Sona', 'Tristana', 'DrMundo', 'Seraphine', 'Viktor', 'Ornn', 'Cassiopeia', 'Garen', 'Yorick', 'Maokai', 'Sivir', 'Zeri', 'Tryndamere', 'Karma', 'Renata', 'Kassadin', 'Katarina', 'Vex', 'Corki', 'Amumu', 'Poppy'

In [11]:
img_rows=256
img_cols=256

In [12]:
num_channel=1

In [13]:
from sklearn import preprocessing

img_data_list=[]

for dataset in data_dir_list: # boucle sur les 162 repertoires
    img_list=os.listdir(data_path+'/'+ dataset)  # 

    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        input_img_raw=  cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img_grey= cv2.cvtColor(input_img_raw, cv2.COLOR_BGR2GRAY)
        input_img_flatten=cv2.resize(input_img_grey,(256,256)).flatten()
        img_data_list.append(input_img_flatten)

        
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')

img_data_scaled = preprocessing.scale(img_data)
print (img_data_scaled.shape)

print (np.mean(img_data_scaled))
print (np.std(img_data_scaled))

print (img_data_scaled.mean(axis=0))
print (img_data_scaled.std(axis=0)) 

Loaded the images of dataset-LeeSin

Loaded the images of dataset-Rakan

Loaded the images of dataset-Diana

Loaded the images of dataset-Vayne

Loaded the images of dataset-MasterYi

Loaded the images of dataset-Nilah

Loaded the images of dataset-Sion

Loaded the images of dataset-Zed

Loaded the images of dataset-Talon

Loaded the images of dataset-Akshan

Loaded the images of dataset-Zoe

Loaded the images of dataset-Akali

Loaded the images of dataset-Bard

Loaded the images of dataset-Riven

Loaded the images of dataset-Vi

Loaded the images of dataset-Ashe

Loaded the images of dataset-Syndra

Loaded the images of dataset-Rell

Loaded the images of dataset-Gragas

Loaded the images of dataset-Darius

Loaded the images of dataset-Malzahar

Loaded the images of dataset-Soraka

Loaded the images of dataset-Gwen

Loaded the images of dataset-Graves

Loaded the images of dataset-Qiyana

Loaded the images of dataset-Azir

Loaded the images of dataset-Viego

Loaded the images of datase

/opt/anaconda3/envs/TP1/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/opt/anaconda3/envs/TP1/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


(1620, 65536)
3.669411e-08
1.0000025
[-1.8838012e-08 -1.0743554e-08  2.1192763e-08 ...  3.7970366e-08
 -1.9647457e-08 -1.6328730e-07]
[1.0000006 1.0000004 0.9999987 ... 1.0000005 1.0000026 1.0000037]


In [77]:
img_data.shape


(1620, 65536)

In [14]:
img_data_reshape=img_data_scaled.reshape(img_data.shape[0],
                                        img_rows,img_cols,
                                        num_channel)
print (img_data_reshape.shape)

(1620, 256, 256, 1)


In [15]:
num_of_samples = img_data_reshape.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')
for i in range(0,num_classes):
    labels[i*10:(i+1)*10]=i

names = data_dir_list

In [16]:
Y = np_utils.to_categorical(labels, num_classes)
print(Y)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [17]:
X_train, X_test, y_train, y_test = train_test_split(img_data_reshape, Y, test_size=0.2, random_state=2)
print(X_train.shape)


(1296, 256, 256, 1)


In [18]:
input_shape=img_data_reshape[0].shape # (128, 128, 1)
input_shape

(256, 256, 1)

In [19]:
my_first_cnn = Sequential()
my_first_cnn.add(Conv2D(256,(3, 3), strides=1, padding='same',input_shape=input_shape))
my_first_cnn.add(Activation('relu'))
my_first_cnn.add(Conv2D(256,(3, 3), strides=1, padding='same'))
my_first_cnn.add(MaxPooling2D(pool_size=(2, 2)))
my_first_cnn.add(Dropout(0.5))
my_first_cnn.add(Flatten())
my_first_cnn.add(Dense(256, activation='relu'))
my_first_cnn.add(Dropout(0.5))
my_first_cnn.add(Dense(162, activation='softmax'))

my_first_cnn.summary()

: 

: 

In [84]:
from keras.optimizers import SGD, RMSprop, Adam, Adadelta
adam = Adam(lr=0.01)
my_first_cnn.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history_adam = my_first_cnn.fit(X_train, y_train, batch_size=16, epochs=10, verbose=1, validation_data=(X_test, y_test))
 

/opt/anaconda3/envs/TP1/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/10
